<a href="https://colab.research.google.com/github/dagmaros27/AIMS_Notebooks/blob/main/CUDA_Practical_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CUDA Programming on NVIDIA GPUs**

# **Practical 3**

Again make sure the correct Runtime is being used, by clicking on the Runtime option at the top, then "Change runtime type", and selecting an appropriate GPU such as the T4.

Then verify with the instruction below the details of the GPU which is available to you.  

In [1]:
!nvidia-smi


Sat Jan 24 07:19:24 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

---

First we upload two header files from the course webpage.

In [2]:
!wget https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_cuda.h
!wget https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_string.h


--2026-01-24 07:19:28--  https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_cuda.h
Resolving people.maths.ox.ac.uk (people.maths.ox.ac.uk)... 129.67.184.129, 2001:630:441:201::8143:b881
Connecting to people.maths.ox.ac.uk (people.maths.ox.ac.uk)|129.67.184.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27832 (27K) [text/x-chdr]
Saving to: ‘helper_cuda.h’

helper_cuda.h       100%[===================>]  27.18K  --.-KB/s    in 0.1s    

2026-01-24 07:19:29 (200 KB/s) - ‘helper_cuda.h’ saved [27832/27832]

--2026-01-24 07:19:29--  https://people.maths.ox.ac.uk/gilesm/cuda/headers/helper_string.h
Resolving people.maths.ox.ac.uk (people.maths.ox.ac.uk)... 129.67.184.129, 2001:630:441:201::8143:b881
Connecting to people.maths.ox.ac.uk (people.maths.ox.ac.uk)|129.67.184.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14875 (15K) [text/x-chdr]
Saving to: ‘helper_string.h’

helper_string.h     100%[===================>]  14.53K 



---

The next step is to create the file laplace3d.cu which includes within it a reference C++ routine against which the CUDA results are compared.

In [13]:
%%writefile laplace3d.cu

////////////////////////////////////////////////////////////////////////
//
// Program to solve Laplace equation on a regular 3D grid
//
////////////////////////////////////////////////////////////////////////

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// kernel function
////////////////////////////////////////////////////////////////////////

__global__ void GPU_laplace3d(long long NX, long long NY, long long NZ,
                              const float* __restrict__ d_u1,
                                    float* __restrict__ d_u2)
{
  long long i, j, k, indg, IOFF, JOFF, KOFF;
  float     u2, sixth=1.0f/6.0f;

  //
  // define global indices and array offsets
  //

  i    = threadIdx.x + blockIdx.x*blockDim.x;
  j    = threadIdx.y + blockIdx.y*blockDim.y;
  indg = i + j*NX;

  IOFF = 1;
  JOFF = NX;
  KOFF = NX*NY;

  if ( i>=0 && i<=NX-1 && j>=0 && j<=NY-1 ) {

    for (k=0; k<NZ; k++) {

      if (i==0 || i==NX-1 || j==0 || j==NY-1 || k==0 || k==NZ-1) {
        u2 = d_u1[indg];  // Dirichlet b.c.'s
      }
      else {
        u2 = ( d_u1[indg-IOFF] + d_u1[indg+IOFF]
             + d_u1[indg-JOFF] + d_u1[indg+JOFF]
             + d_u1[indg-KOFF] + d_u1[indg+KOFF] ) * sixth;
      }
      d_u2[indg] = u2;

      indg += KOFF;
    }
  }
}


////////////////////////////////////////////////////////////////////////
// Gold routine -- reference C++ code
////////////////////////////////////////////////////////////////////////

void Gold_laplace3d(long long NX, long long NY, long long NZ, float* u1, float* u2)
{
  long long i, j, k, ind;
  float     sixth=1.0f/6.0f;  // predefining this improves performance more than 10%

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {   // i loop innermost for sequential memory access
	      ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1) {
          u2[ind] = u1[ind];          // Dirichlet b.c.'s
        }
        else {
          u2[ind] = ( u1[ind-1    ] + u1[ind+1    ]
                    + u1[ind-NX   ] + u1[ind+NX   ]
                    + u1[ind-NX*NY] + u1[ind+NX*NY] ) * sixth;
        }
      }
    }
  }
}

////////////////////////////////////////////////////////////////////////
// Main program
////////////////////////////////////////////////////////////////////////

int main(int argc, const char **argv){

  int       NX=1024, NY=1024, NZ=1024, REPEAT=20,
            BLOCK_X, BLOCK_Y, bx, by, i, j, k;
  float    *h_u1, *h_u2, *h_foo,
           *d_u1, *d_u2, *d_foo;

  size_t    ind, bytes = sizeof(float) * NX*NY*NZ;

  printf("Grid dimensions: %d x %d x %d \n\n", NX, NY, NZ);

  // initialise card

  findCudaDevice(argc, argv);

  // initialise CUDA timing

  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  // allocate memory for arrays

  h_u1 = (float *)malloc(bytes);
  h_u2 = (float *)malloc(bytes);
  checkCudaErrors( cudaMalloc((void **)&d_u1, bytes) );
  checkCudaErrors( cudaMalloc((void **)&d_u2, bytes) );

  // initialise u1

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1)
          h_u1[ind] = 1.0f;           // Dirichlet b.c.'s
        else
          h_u1[ind] = 0.0f;
      }
    }
  }

  // copy u1 to device

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(d_u1, h_u1, bytes,
                              cudaMemcpyHostToDevice) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u1 to device: %.1f (ms) \n\n", milli);

  // Gold treatment

  //cudaEventRecord(start);
  //for (i=0; i<REPEAT; i++) {
  //  Gold_laplace3d(NX, NY, NZ, h_u1, h_u2);
  //  h_foo = h_u1; h_u1 = h_u2; h_u2 = h_foo;   // swap h_u1 and h_u2
  //}

  //cudaEventRecord(stop);
  //cudaEventSynchronize(stop);
  //cudaEventElapsedTime(&milli, start, stop);
  //printf("%dx Gold_laplace3d: %.1f (ms) \n\n", REPEAT, milli);

  // Set up the execution configuration

  BLOCK_X = 64; // number of threads
  BLOCK_Y = 16; // in each direction

  bx = 1 + (NX-1)/BLOCK_X; // number of blocks
  by = 1 + (NY-1)/BLOCK_Y; // in each direction

  dim3 dimGrid(bx,by);
  dim3 dimBlock(BLOCK_X,BLOCK_Y);

  // Execute GPU kernel

  cudaEventRecord(start);

  for (i=0; i<REPEAT; i++) {
    GPU_laplace3d<<<dimGrid, dimBlock>>>(NX, NY, NZ, d_u1, d_u2);
    getLastCudaError("GPU_laplace3d execution failed\n");

    d_foo = d_u1; d_u1 = d_u2; d_u2 = d_foo;   // swap d_u1 and d_u2
  }

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Block dimensions: %d x %d \n", BLOCK_X,BLOCK_Y);
  printf("%dx GPU_laplace3d: %.1f (ms) \n\n", REPEAT, milli);

  // Read back GPU results

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(h_u2, d_u1, bytes, cudaMemcpyDeviceToHost) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u2 to host: %.1f (ms) \n\n", milli);

  // error check

  float err = 0.0;

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;
        err += (h_u1[ind]-h_u2[ind])*(h_u1[ind]-h_u2[ind]);
      }
    }
  }

  printf("rms error = %f \n",sqrt(err/ (float)(NX*NY*NZ)));

 // Release GPU and CPU memory

  checkCudaErrors( cudaFree(d_u1) );
  checkCudaErrors( cudaFree(d_u2) );
  free(h_u1);
  free(h_u2);

  cudaDeviceReset();
}


Writing laplace3d.cu



---

We can now compile and run the executable.


In [17]:
!nvcc laplace3d.cu -o laplace3d -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z13GPU_laplace3dxxxPKfPf' for 'sm_70'
ptxas info    : Function properties for _Z13GPU_laplace3dxxxPKfPf
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 64 registers, 392 bytes cmem[0]


In [15]:
!./laplace3d

Grid dimensions: 1024 x 1024 x 1024 

GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 968.0 (ms) 

Block dimensions: 64 x 16 
20x GPU_laplace3d: 1210.7 (ms) 

Copy u2 to host: 2811.4 (ms) 

rms error = 0.066320 


```
Grid dimensions: 1024 x 1024 x 1024

GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 938.8 (ms)

Block  2 x  2  ->  5071.906 ms
Block  2 x  4  ->  5656.572 ms
Block  2 x  8  ->  10098.975 ms
Block  2 x 16  ->  18532.633 ms
Block  2 x 24  ->  22519.248 ms
Block  2 x 32  ->  23792.822 ms
Block  2 x 64  ->  25030.516 ms
Block  4 x  2  ->  5273.936 ms
Block  4 x  4  ->  6782.370 ms
Block  4 x  8  ->  7705.290 ms
Block  4 x 16  ->  11683.622 ms
Block  4 x 24  ->  11806.559 ms
Block  4 x 32  ->  11969.221 ms
Block  4 x 64  ->  12230.748 ms
Block  8 x  2  ->  2203.408 ms
Block  8 x  4  ->  2161.137 ms
Block  8 x  8  ->  3006.282 ms
Block  8 x 16  ->  3431.023 ms
Block  8 x 24  ->  3469.136 ms
Block  8 x 32  ->  3829.409 ms
Block  8 x 64  ->  4184.480 ms
Block 16 x  2  ->  1661.684 ms
Block 16 x  4  ->  1852.686 ms
Block 16 x  8  ->  1842.868 ms
Block 16 x 16  ->  1930.509 ms
Block 16 x 24  ->  1839.395 ms
Block 16 x 32  ->  2090.338 ms
Block 16 x 64  ->  2002.745 ms
Block 24 x  2  ->  2131.153 ms
Block 24 x  4  ->  1809.894 ms
Block 24 x  8  ->  1870.719 ms
Block 24 x 16  ->  1694.615 ms
Block 24 x 24  ->  1728.848 ms
Block 24 x 32  ->  1756.730 ms
Block 32 x  2  ->  1465.997 ms
Block 32 x  4  ->  1280.623 ms
Block 32 x  8  ->  1237.430 ms
Block 32 x 16  ->  1243.163 ms
Block 32 x 24  ->  1272.825 ms
Block 32 x 32  ->  1270.032 ms
Block 64 x  2  ->  1483.745 ms
Block 64 x  4  ->  1325.004 ms
Block 64 x  8  ->  1231.041 ms
Block 64 x 16  ->  1191.235 ms

==== BEST CONFIGURATION ====
Block: 64 x 16   Time: 1191.235 ms
Copy u2 to host: 2908.7 (ms)

rms error = 0.125000




---
By going back to the previous code block you can modify the code to complete the initial Practical 3 exercises. Remember to first make your own copy of the notebook so that you are able to edit it.

For students doing this as an assignment to be assessed, you should again add your name to the title of the notebook (as in "Practical 3 -- Mike Giles.ipynb"), make it shared (see the Share option in the top-right corner) and provide the shared link as the submission mechanism.


---

For the later parts of Practical 3, the instructions below create, compile and execute a second version of the code in which each CUDA thread computes the value for just one 3D grid point.

In [3]:
%%writefile laplace3d_new.cu


//
// Program to solve Laplace equation on a regular 3D grid
//

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// kernel function
////////////////////////////////////////////////////////////////////////

__global__ void GPU_laplace3d(long long NX, long long NY, long long NZ,
	         	      const float* __restrict__ d_u1,
			            float* __restrict__ d_u2)
{
  long long i, j, k, indg, IOFF, JOFF, KOFF;
  float     u2, sixth=1.0f/6.0f;

  //
  // define global indices and array offsets
  //

  i    = threadIdx.x + blockIdx.x*blockDim.x;
  j    = threadIdx.y + blockIdx.y*blockDim.y;
  k    = threadIdx.z + blockIdx.z*blockDim.z;

  IOFF = 1;
  JOFF = NX;
  KOFF = NX*NY;

  indg = i + j*JOFF + k*KOFF;

  if (i>=0 && i<=NX-1 && j>=0 && j<=NY-1 && k>=0 && k<=NZ-1) {
    if (i==0 || i==NX-1 || j==0 || j==NY-1 || k==0 || k==NZ-1) {
      u2 = d_u1[indg];  // Dirichlet b.c.'s
    }
    else {
      u2 = ( d_u1[indg-IOFF] + d_u1[indg+IOFF]
           + d_u1[indg-JOFF] + d_u1[indg+JOFF]
           + d_u1[indg-KOFF] + d_u1[indg+KOFF] ) * sixth;
    }
    d_u2[indg] = u2;
  }
}

////////////////////////////////////////////////////////////////////////
// Gold routine -- reference C++ code
////////////////////////////////////////////////////////////////////////

void Gold_laplace3d(long long NX, long long NY, long long NZ, float* u1, float* u2)
{
  long long i, j, k, ind;
  float     sixth=1.0f/6.0f;  // predefining this improves performance more than 10%

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {   // i loop innermost for sequential memory access
	      ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1) {
          u2[ind] = u1[ind];          // Dirichlet b.c.'s
        }
        else {
          u2[ind] = ( u1[ind-1    ] + u1[ind+1    ]
                    + u1[ind-NX   ] + u1[ind+NX   ]
                    + u1[ind-NX*NY] + u1[ind+NX*NY] ) * sixth;
        }
      }
    }
  }
}


////////////////////////////////////////////////////////////////////////
// Main program
////////////////////////////////////////////////////////////////////////

int main(int argc, const char **argv){

  int       NX=1024, NY=1024, NZ=1024, REPEAT=1,
            BLOCK_X, BLOCK_Y, BLOCK_Z,bx, by, bz, i, j, k;
  float    *h_u1, *h_u2,
           *d_u1, *d_u2, *d_foo;

  size_t    ind, bytes = sizeof(float) * NX*NY*NZ;

  printf("Grid dimensions: %d x %d x %d \n\n", NX, NY, NZ);

  // initialise card

  findCudaDevice(argc, argv);

  // initialise CUDA timing

  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  // allocate memory for arrays

  h_u1 = (float *)malloc(bytes);
  h_u2 = (float *)malloc(bytes);
  checkCudaErrors( cudaMalloc((void **)&d_u1, bytes) );
  checkCudaErrors( cudaMalloc((void **)&d_u2, bytes) );

  // initialise u1

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1)
          h_u1[ind] = 1.0f;           // Dirichlet b.c.'s
        else
          h_u1[ind] = 0.0f;
      }
    }
  }

  // copy u1 to device

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(d_u1, h_u1, bytes,
                              cudaMemcpyHostToDevice) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u1 to device: %.1f (ms) \n\n", milli);

  // Gold treatment

//   cudaEventRecord(start);
//   for (i=0; i<REPEAT; i++) {
//     Gold_laplace3d(NX, NY, NZ, h_u1, h_u2);
//     h_foo = h_u1; h_u1 = h_u2; h_u2 = h_foo;   // swap h_u1 and h_u2
//   }

//   cudaEventRecord(stop);
//   cudaEventSynchronize(stop);
//   cudaEventElapsedTime(&milli, start, stop);
//   printf("%dx Gold_laplace3d: %.1f (ms) \n\n", REPEAT, milli);

  // Set up the execution configuration

  BLOCK_X =32; // number of threads
  BLOCK_Y = 2; // in each direction
  BLOCK_Z = 8; // of 3D thread block

  bx = 1 + (NX-1)/BLOCK_X; // number of blocks
  by = 1 + (NY-1)/BLOCK_Y; // in each direction
  bz = 1 + (NZ-1)/BLOCK_Z; // of 3D grid of blocks

  dim3 dimGrid(bx,by,bz);
  dim3 dimBlock(BLOCK_X,BLOCK_Y,BLOCK_Z);

// Execute GPU kernel

cudaEventRecord(start);

for (i=0; i<REPEAT; i++) {
  GPU_laplace3d<<<dimGrid, dimBlock>>>(NX, NY, NZ, d_u1, d_u2);
  getLastCudaError("GPU_laplace3d execution failed\n");

  d_foo = d_u1; d_u1 = d_u2; d_u2 = d_foo;   // swap d_u1 and d_u2
}

cudaEventRecord(stop);
cudaEventSynchronize(stop);
cudaEventElapsedTime(&milli, start, stop);
printf("Block dimensions: %d x %d x %d\n", BLOCK_X,BLOCK_Y,BLOCK_Z);
printf("%dx GPU_laplace3d_new: %.1f (ms) \n", REPEAT, milli);

// Calculate bandwidth per iteration
float time_per_iter = milli / (float)REPEAT / 1000.0f; // convert to seconds
float bytes_per_iter = 2.0f * (float)bytes; // read d_u1 + write d_u2
float gb_per_iter = bytes_per_iter / 1e9f; // convert to GB
float effective_bw = gb_per_iter / time_per_iter;

printf("\nBandwidth Analysis:\n");
printf("Time per iteration: %.3f ms\n", milli / (float)REPEAT);
printf("Data per iteration: %.3f GB\n", gb_per_iter);
printf("Effective Bandwidth: %.2f GB/s\n\n", effective_bw);


  // Read back GPU results

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(h_u2, d_u1, bytes, cudaMemcpyDeviceToHost) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u2 to host: %.1f (ms) \n\n", milli);


  // error check

  float err = 0.0;

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;
        err += (h_u1[ind]-h_u2[ind])*(h_u1[ind]-h_u2[ind]);
      }
    }
  }

  printf("rms error = %f \n",sqrt(err/ (float)(NX*NY*NZ)));

 // Release GPU and CPU memory

  checkCudaErrors( cudaFree(d_u1) );
  checkCudaErrors( cudaFree(d_u2) );
  free(h_u1);
  free(h_u2);

  cudaDeviceReset();
}


Writing laplace3d_new.cu


In [18]:
!nvcc laplace3d_new.cu -o laplace3d_new -I. -lineinfo -arch=sm_70 --ptxas-options=-v --use_fast_math -lcudart

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z13GPU_laplace3dxxxPKfPf' for 'sm_70'
ptxas info    : Function properties for _Z13GPU_laplace3dxxxPKfPf
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 21 registers, 392 bytes cmem[0]


In [12]:
!./laplace3d_new

Grid dimensions: 1024 x 1024 x 1024 

GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 1008.8 (ms) 

Block dimensions: 32 x 2 x 8
20x GPU_laplace3d_new: 877.4 (ms) 

Bandwidth Analysis:
Time per iteration: 43.869 ms
Data per iteration: 8.590 GB
Effective Bandwidth: 195.81 GB/s

Copy u2 to host: 2860.0 (ms) 

rms error = 0.066320 


```
Grid dimensions: 1024 x 1024 x 1024

GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 975.4 (ms)

Block 2 x 2 x 2  ->  4028.775 ms
Block 2 x 2 x 4  ->  3152.670 ms
Block 2 x 2 x 8  ->  3131.752 ms
Block 2 x 2 x 16  ->  3210.297 ms
Block 2 x 2 x 24  ->  3272.550 ms
Block 2 x 2 x 32  ->  3401.198 ms
Block 2 x 2 x 64  ->  3515.413 ms
Block 2 x 4 x 2  ->  3418.371 ms
Block 2 x 4 x 4  ->  3346.674 ms
Block 2 x 4 x 8  ->  3358.173 ms
Block 2 x 4 x 16  ->  3472.517 ms
Block 2 x 4 x 24  ->  3516.457 ms
Block 2 x 4 x 32  ->  3552.722 ms
Block 2 x 4 x 64  ->  3617.280 ms
Block 2 x 8 x 2  ->  3561.329 ms
Block 2 x 8 x 4  ->  3499.206 ms
Block 2 x 8 x 8  ->  3572.329 ms
Block 2 x 8 x 16  ->  3598.354 ms
Block 2 x 8 x 24  ->  3623.103 ms
Block 2 x 8 x 32  ->  3678.904 ms
Block 2 x 8 x 64  ->  3999.664 ms
Block 2 x 16 x 2  ->  3843.965 ms
Block 2 x 16 x 4  ->  3857.678 ms
Block 2 x 16 x 8  ->  3827.151 ms
Block 2 x 16 x 16  ->  3827.921 ms
Block 2 x 16 x 24  ->  3905.278 ms
Block 2 x 16 x 32  ->  4019.314 ms
Block 2 x 24 x 2  ->  3925.576 ms
Block 2 x 24 x 4  ->  3861.359 ms
Block 2 x 24 x 8  ->  3756.844 ms
Block 2 x 24 x 16  ->  3815.725 ms
Block 2 x 32 x 2  ->  3971.308 ms
Block 2 x 32 x 4  ->  3797.692 ms
Block 2 x 32 x 8  ->  3720.925 ms
Block 2 x 32 x 16  ->  3882.498 ms
Block 2 x 64 x 2  ->  3944.873 ms
Block 2 x 64 x 4  ->  3827.793 ms
Block 2 x 64 x 8  ->  3975.903 ms
Block 4 x 2 x 2  ->  2564.700 ms
Block 4 x 2 x 4  ->  2082.920 ms
Block 4 x 2 x 8  ->  2344.964 ms
Block 4 x 2 x 16  ->  2431.607 ms
Block 4 x 2 x 24  ->  2427.282 ms
Block 4 x 2 x 32  ->  2463.807 ms
Block 4 x 2 x 64  ->  2559.384 ms
Block 4 x 4 x 2  ->  2180.882 ms
Block 4 x 4 x 4  ->  2439.271 ms
Block 4 x 4 x 8  ->  2437.106 ms
Block 4 x 4 x 16  ->  2417.604 ms
Block 4 x 4 x 24  ->  2418.261 ms
Block 4 x 4 x 32  ->  2469.730 ms
Block 4 x 4 x 64  ->  2583.128 ms
Block 4 x 8 x 2  ->  2514.749 ms
Block 4 x 8 x 4  ->  2391.132 ms
Block 4 x 8 x 8  ->  2301.352 ms
Block 4 x 8 x 16  ->  2331.021 ms
Block 4 x 8 x 24  ->  2490.983 ms
Block 4 x 8 x 32  ->  2397.463 ms
Block 4 x 16 x 2  ->  2560.818 ms
Block 4 x 16 x 4  ->  2363.711 ms
Block 4 x 16 x 8  ->  2346.293 ms
Block 4 x 16 x 16  ->  2387.619 ms
Block 4 x 24 x 2  ->  2544.709 ms
Block 4 x 24 x 4  ->  2371.431 ms
Block 4 x 24 x 8  ->  2456.529 ms
Block 4 x 32 x 2  ->  2563.068 ms
Block 4 x 32 x 4  ->  2425.422 ms
Block 4 x 32 x 8  ->  2417.038 ms
Block 4 x 64 x 2  ->  2679.237 ms
Block 4 x 64 x 4  ->  2535.379 ms
Block 8 x 2 x 2  ->  1446.926 ms
Block 8 x 2 x 4  ->  1466.336 ms
Block 8 x 2 x 8  ->  1447.469 ms
Block 8 x 2 x 16  ->  1394.011 ms
Block 8 x 2 x 24  ->  1424.741 ms
Block 8 x 2 x 32  ->  1405.135 ms
Block 8 x 2 x 64  ->  1591.388 ms
Block 8 x 4 x 2  ->  1448.647 ms
Block 8 x 4 x 4  ->  1391.279 ms
Block 8 x 4 x 8  ->  1383.117 ms
Block 8 x 4 x 16  ->  1387.184 ms
Block 8 x 4 x 24  ->  1707.434 ms
Block 8 x 4 x 32  ->  1621.762 ms
Block 8 x 8 x 2  ->  1465.731 ms
Block 8 x 8 x 4  ->  1393.674 ms
Block 8 x 8 x 8  ->  1361.934 ms
Block 8 x 8 x 16  ->  1571.893 ms
Block 8 x 16 x 2  ->  1538.924 ms
Block 8 x 16 x 4  ->  1421.554 ms
Block 8 x 16 x 8  ->  1567.387 ms
Block 8 x 24 x 2  ->  1561.719 ms
Block 8 x 24 x 4  ->  1705.338 ms
Block 8 x 32 x 2  ->  1599.737 ms
Block 8 x 32 x 4  ->  1604.334 ms
Block 8 x 64 x 2  ->  1726.994 ms
Block 16 x 2 x 2  ->  1305.208 ms
Block 16 x 2 x 4  ->  1230.835 ms
Block 16 x 2 x 8  ->  1173.293 ms
Block 16 x 2 x 16  ->  1152.732 ms
Block 16 x 2 x 24  ->  1383.883 ms
Block 16 x 2 x 32  ->  1264.628 ms
Block 16 x 4 x 2  ->  1313.274 ms
Block 16 x 4 x 4  ->  1203.122 ms
Block 16 x 4 x 8  ->  1150.054 ms
Block 16 x 4 x 16  ->  1269.721 ms
Block 16 x 8 x 2  ->  1268.688 ms
Block 16 x 8 x 4  ->  1182.125 ms
Block 16 x 8 x 8  ->  1275.988 ms
Block 16 x 16 x 2  ->  1265.813 ms
Block 16 x 16 x 4  ->  1311.232 ms
Block 16 x 24 x 2  ->  1469.645 ms
Block 16 x 32 x 2  ->  1377.120 ms
Block 24 x 2 x 2  ->  1350.105 ms
Block 24 x 2 x 4  ->  1236.255 ms
Block 24 x 2 x 8  ->  1187.500 ms
Block 24 x 2 x 16  ->  1329.991 ms
Block 24 x 4 x 2  ->  1310.304 ms
Block 24 x 4 x 4  ->  1218.322 ms
Block 24 x 4 x 8  ->  1337.341 ms
Block 24 x 8 x 2  ->  1274.751 ms
Block 24 x 8 x 4  ->  1364.607 ms
Block 24 x 16 x 2  ->  1420.832 ms
Block 32 x 2 x 2  ->  1186.519 ms
Block 32 x 2 x 4  ->  1099.031 ms
Block 32 x 2 x 8  ->  1036.686 ms
Block 32 x 2 x 16  ->  1096.112 ms
Block 32 x 4 x 2  ->  1154.237 ms
Block 32 x 4 x 4  ->  1070.278 ms
Block 32 x 4 x 8  ->  1109.132 ms
Block 32 x 8 x 2  ->  1134.169 ms
Block 32 x 8 x 4  ->  1150.491 ms
Block 32 x 16 x 2  ->  1193.914 ms
Block 64 x 2 x 2  ->  1163.561 ms
Block 64 x 2 x 4  ->  1066.783 ms
Block 64 x 2 x 8  ->  1079.992 ms
Block 64 x 4 x 2  ->  1128.703 ms
Block 64 x 4 x 4  ->  1103.254 ms
Block 64 x 8 x 2  ->  1160.750 ms

==== BEST CONFIGURATION ====
Block: 32 x 2 x 8   Time: 1036.686 ms
Copy u2 to host: 2831.6 (ms)

rms error = 0.125000



---

The next instructions check how many fp32 and integer instructions are performed by the two versions

In [19]:
!ncu --metrics "smsp__sass_thread_inst_executed_op_fp32_pred_on.sum,smsp__sass_thread_inst_executed_op_integer_pred_on.sum" ./laplace3d
!ncu --metrics "smsp__sass_thread_inst_executed_op_fp32_pred_on.sum,smsp__sass_thread_inst_executed_op_integer_pred_on.sum" ./laplace3d_new

Grid dimensions: 1024 x 1024 x 1024 

==PROF== Connected to process 9820 (/content/laplace3d)
GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 958.5 (ms) 

==PROF== Profiling "GPU_laplace3d" - 0: 0%....50%....100% - 1 pass
Block dimensions: 64 x 16 
1x GPU_laplace3d: 1938.9 (ms) 

Copy u2 to host: 3281.5 (ms) 

rms error = 0.013208 
==PROF== Disconnected from process 9820
[9820] laplace3d@127.0.0.1
  GPU_laplace3d(long long, long long, long long, const float *, float *) (16, 64, 1)x(64, 16, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: Command line profiler metrics
    ------------------------------------------------------ ----------- --------------
    Metric Name                                            Metric Unit   Metric Value
    ------------------------------------------------------ ----------- --------------
    smsp__sass_thread_inst_executed_op_fp32_pred_on.sum           inst  6,404,775,888
    smsp__sass_thread_inst_executed_op_integer_pred_o

In [20]:
!ncu ./laplace3d
!ncu ./laplace3d_new

Grid dimensions: 1024 x 1024 x 1024 

==PROF== Connected to process 10242 (/content/laplace3d)
GPU Device 0: "Turing" with compute capability 7.5

Copy u1 to device: 969.2 (ms) 

==PROF== Profiling "GPU_laplace3d" - 0: 0%
==WARNING== Backing up device memory in system memory. Kernel replay might be slow. Consider using "--replay-mode application" to avoid memory save-and-restore.
....50%....100% - 9 passes
Block dimensions: 64 x 16 
1x GPU_laplace3d: 3066.6 (ms) 

Copy u2 to host: 3519.4 (ms) 

rms error = 0.013208 
==PROF== Disconnected from process 10242
[10242] laplace3d@127.0.0.1
  GPU_laplace3d(long long, long long, long long, const float *, float *) (16, 64, 1)x(64, 16, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- -------------
    Metric Name             Metric Unit  Metric Value
    ----------------------- ----------- -------------
    DRAM Frequency                  Ghz          4.99
    SM Frequen

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
%%writefile laplace3d.cu


////////////////////////////////////////////////////////////////////////
//
// Program to solve Laplace equation on a regular 3D grid
//
////////////////////////////////////////////////////////////////////////

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// kernel function
////////////////////////////////////////////////////////////////////////

__global__ void GPU_laplace3d(long long NX, long long NY, long long NZ,
                              const float* __restrict__ d_u1,
                                    float* __restrict__ d_u2)
{
  long long i, j, k, indg, IOFF, JOFF, KOFF;
  float     u2, sixth=1.0f/6.0f;

  //
  // define global indices and array offsets
  //

  i    = threadIdx.x + blockIdx.x*blockDim.x;
  j    = threadIdx.y + blockIdx.y*blockDim.y;
  indg = i + j*NX;

  IOFF = 1;
  JOFF = NX;
  KOFF = NX*NY;

  if ( i>=0 && i<=NX-1 && j>=0 && j<=NY-1 ) {

    for (k=0; k<NZ; k++) {

      if (i==0 || i==NX-1 || j==0 || j==NY-1 || k==0 || k==NZ-1) { // bound elements
        u2 = d_u1[indg];  // Dirichlet b.c.'s
      }
      else {
        u2 = ( d_u1[indg-IOFF] + d_u1[indg+IOFF]
             + d_u1[indg-JOFF] + d_u1[indg+JOFF]
             + d_u1[indg-KOFF] + d_u1[indg+KOFF] ) * sixth;
      }
      d_u2[indg] = u2;

      indg += KOFF;
    }
  }
}


////////////////////////////////////////////////////////////////////////
// Gold routine -- reference C++ code
////////////////////////////////////////////////////////////////////////

void Gold_laplace3d(long long NX, long long NY, long long NZ, float* u1, float* u2)
{
  long long i, j, k, ind;
  float     sixth=1.0f/6.0f;  // predefining this improves performance more than 10%

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {   // i loop innermost for sequential memory access
	      ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1) {
          u2[ind] = u1[ind];          // Dirichlet b.c.'s
        }
        else {
          u2[ind] = ( u1[ind-1    ] + u1[ind+1    ]
                    + u1[ind-NX   ] + u1[ind+NX   ]
                    + u1[ind-NX*NY] + u1[ind+NX*NY] ) * sixth;
        }
      }
    }
  }
}

////////////////////////////////////////////////////////////////////////
// Main program
////////////////////////////////////////////////////////////////////////

int main(int argc, const char **argv){

  int       NX=1024, NY=1024, NZ=1024, REPEAT=20,
          i, j, k;
  float    *h_u1, *h_u2,
         *d_u1, *d_u2, *d_foo;


  size_t    ind, bytes = sizeof(float) * NX*NY*NZ;

  printf("Grid dimensions: %d x %d x %d \n\n", NX, NY, NZ);

  // initialise card

  findCudaDevice(argc, argv);

  // initialise CUDA timing

  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  // allocate memory for arrays

  h_u1 = (float *)malloc(bytes);
  h_u2 = (float *)malloc(bytes);
  checkCudaErrors( cudaMalloc((void **)&d_u1, bytes) );
  checkCudaErrors( cudaMalloc((void **)&d_u2, bytes) );

  // initialise u1

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1)
          h_u1[ind] = 1.0f;           // Dirichlet b.c.'s
        else
          h_u1[ind] = 0.0f;
      }
    }
  }

  // copy u1 to device

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(d_u1, h_u1, bytes,
                              cudaMemcpyHostToDevice) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u1 to device: %.1f (ms) \n\n", milli);

  // Gold treatment

//   cudaEventRecord(start);
//   for (i=0; i<REPEAT; i++) {
//     Gold_laplace3d(NX, NY, NZ, h_u1, h_u2);
//     h_foo = h_u1; h_u1 = h_u2; h_u2 = h_foo;   // swap h_u1 and h_u2
//   }

//   cudaEventRecord(stop);
//   cudaEventSynchronize(stop);
//   cudaEventElapsedTime(&milli, start, stop);
//   printf("%dx Gold_laplace3d: %.1f (ms) \n\n", REPEAT, milli);

  // Set up the execution configuration

  int block_sizes[] = {2, 4, 8, 16, 24, 32, 64};   // try common warp-friendly sizes
  int nSizes = sizeof(block_sizes)/sizeof(block_sizes[0]);

  float bestTime = 1e30;
  int bestBX = 0, bestBY = 0;

  for (int ix = 0; ix < nSizes; ix++) {
  for (int iy = 0; iy < nSizes; iy++) {

    int BLOCK_X = block_sizes[ix];
    int BLOCK_Y = block_sizes[iy];

    // Skip illegal configurations (>1024 threads per block)
    if (BLOCK_X * BLOCK_Y > 1024) continue;

    int bx = 1 + (NX-1)/BLOCK_X;
    int by = 1 + (NY-1)/BLOCK_Y;

    dim3 dimGrid(bx, by);
    dim3 dimBlock(BLOCK_X, BLOCK_Y);

    cudaDeviceSynchronize();

    cudaEventRecord(start);

    for (i = 0; i < REPEAT; i++) {
    GPU_laplace3d<<<dimGrid, dimBlock>>>(NX, NY, NZ, d_u1, d_u2);
    getLastCudaError("GPU_laplace3d execution failed\n");

    d_foo = d_u1; d_u1 = d_u2; d_u2 = d_foo;
    }

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milli, start, stop);

    printf("Block %2d x %2d  ->  %.3f ms\n", BLOCK_X, BLOCK_Y, milli);

    if (milli < bestTime) {
    bestTime = milli;
    bestBX = BLOCK_X;
    bestBY = BLOCK_Y;
    }
  }
  }

printf("\n==== BEST CONFIGURATION ====\n");
printf("Block: %d x %d   Time: %.3f ms\n", bestBX, bestBY, bestTime);


  // Read back GPU results

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(h_u2, d_u1, bytes, cudaMemcpyDeviceToHost) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u2 to host: %.1f (ms) \n\n", milli);

  // error check

  float err = 0.0;

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;
        err += (h_u1[ind]-h_u2[ind])*(h_u1[ind]-h_u2[ind]);
      }
    }
  }

  printf("rms error = %f \n",sqrt(err/ (float)(NX*NY*NZ)));

 // Release GPU and CPU memory

  checkCudaErrors( cudaFree(d_u1) );
  checkCudaErrors( cudaFree(d_u2) );
  free(h_u1);
  free(h_u2);

  cudaDeviceReset();
}



Overwriting laplace3d.cu


In [ ]:
%%writefile laplace3d_new.cu


//
// Program to solve Laplace equation on a regular 3D grid
//

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <math.h>

#include <helper_cuda.h>

////////////////////////////////////////////////////////////////////////
// kernel function
////////////////////////////////////////////////////////////////////////

__global__ void GPU_laplace3d(long long NX, long long NY, long long NZ,
	         	      const float* __restrict__ d_u1,
			            float* __restrict__ d_u2)
{
  long long i, j, k, indg, IOFF, JOFF, KOFF;
  float     u2, sixth=1.0f/6.0f;

  //
  // define global indices and array offsets
  //

  i    = threadIdx.x + blockIdx.x*blockDim.x;
  j    = threadIdx.y + blockIdx.y*blockDim.y;
  k    = threadIdx.z + blockIdx.z*blockDim.z;

  IOFF = 1;
  JOFF = NX;
  KOFF = NX*NY;

  indg = i + j*JOFF + k*KOFF;

  if (i>=0 && i<=NX-1 && j>=0 && j<=NY-1 && k>=0 && k<=NZ-1) {
    if (i==0 || i==NX-1 || j==0 || j==NY-1 || k==0 || k==NZ-1) {
      u2 = d_u1[indg];  // Dirichlet b.c.'s
    }
    else {
      u2 = ( d_u1[indg-IOFF] + d_u1[indg+IOFF]
           + d_u1[indg-JOFF] + d_u1[indg+JOFF]
           + d_u1[indg-KOFF] + d_u1[indg+KOFF] ) * sixth;
    }
    d_u2[indg] = u2;
  }
}

////////////////////////////////////////////////////////////////////////
// Gold routine -- reference C++ code
////////////////////////////////////////////////////////////////////////

void Gold_laplace3d(long long NX, long long NY, long long NZ, float* u1, float* u2)
{
  long long i, j, k, ind;
  float     sixth=1.0f/6.0f;  // predefining this improves performance more than 10%

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {   // i loop innermost for sequential memory access
	      ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1) {
          u2[ind] = u1[ind];          // Dirichlet b.c.'s
        }
        else {
          u2[ind] = ( u1[ind-1    ] + u1[ind+1    ]
                    + u1[ind-NX   ] + u1[ind+NX   ]
                    + u1[ind-NX*NY] + u1[ind+NX*NY] ) * sixth;
        }
      }
    }
  }
}


////////////////////////////////////////////////////////////////////////
// Main program
////////////////////////////////////////////////////////////////////////

int main(int argc, const char **argv){

  int       NX=1024, NY=1024, NZ=1024, REPEAT=20,
            BLOCK_X, BLOCK_Y, BLOCK_Z,bx, by, bz, i, j, k;
  float    *h_u1, *h_u2,
           *d_u1, *d_u2, *d_foo;

  size_t    ind, bytes = sizeof(float) * NX*NY*NZ;

  printf("Grid dimensions: %d x %d x %d \n\n", NX, NY, NZ);

  // initialise card

  findCudaDevice(argc, argv);

  // initialise CUDA timing

  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  // allocate memory for arrays

  h_u1 = (float *)malloc(bytes);
  h_u2 = (float *)malloc(bytes);
  checkCudaErrors( cudaMalloc((void **)&d_u1, bytes) );
  checkCudaErrors( cudaMalloc((void **)&d_u2, bytes) );

  // initialise u1

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;

        if (i==0 || i==NX-1 || j==0 || j==NY-1|| k==0 || k==NZ-1)
          h_u1[ind] = 1.0f;           // Dirichlet b.c.'s
        else
          h_u1[ind] = 0.0f;
      }
    }
  }

  // copy u1 to device

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(d_u1, h_u1, bytes,
                              cudaMemcpyHostToDevice) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u1 to device: %.1f (ms) \n\n", milli);

  // Gold treatment

//   cudaEventRecord(start);
//   for (i=0; i<REPEAT; i++) {
//     Gold_laplace3d(NX, NY, NZ, h_u1, h_u2);
//     h_foo = h_u1; h_u1 = h_u2; h_u2 = h_foo;   // swap h_u1 and h_u2
//   }

//   cudaEventRecord(stop);
//   cudaEventSynchronize(stop);
//   cudaEventElapsedTime(&milli, start, stop);
//   printf("%dx Gold_laplace3d: %.1f (ms) \n\n", REPEAT, milli);

  // Set up the execution configuration

  int block_sizes[] = {2, 4, 8, 16, 24, 32, 64};
  int nSizes = sizeof(block_sizes)/sizeof(block_sizes[0]);

  float bestTime = 1e30;
  int bestBX = 0, bestBY = 0, bestBZ = 0;

  for (int ix = 0; ix < nSizes; ix++) {
    for (int iy = 0; iy < nSizes; iy++) {
      for (int iz = 0; iz < nSizes; iz++) {

        BLOCK_X = block_sizes[ix];
        BLOCK_Y = block_sizes[iy];
        BLOCK_Z = block_sizes[iz];

        if (BLOCK_X * BLOCK_Y * BLOCK_Z > 1024) continue;

        bx = 1 + (NX-1)/BLOCK_X;
        by = 1 + (NY-1)/BLOCK_Y;
        bz = 1 + (NZ-1)/BLOCK_Z;

        dim3 dimGrid(bx, by, bz);
        dim3 dimBlock(BLOCK_X, BLOCK_Y, BLOCK_Z);

        cudaDeviceSynchronize();
        cudaEventRecord(start);

        for (i=0; i<REPEAT; i++) {
          GPU_laplace3d<<<dimGrid, dimBlock>>>(NX, NY, NZ, d_u1, d_u2);
          getLastCudaError("GPU_laplace3d execution failed\n");

          d_foo = d_u1; d_u1 = d_u2; d_u2 = d_foo;
        }

        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&milli, start, stop);

        printf("Block %d x %d x %d  ->  %.3f ms\n", BLOCK_X, BLOCK_Y, BLOCK_Z, milli);

        if (milli < bestTime) {
          bestTime = milli;
          bestBX = BLOCK_X; bestBY = BLOCK_Y; bestBZ = BLOCK_Z;
        }
      }
    }
  }

  printf("\n==== BEST CONFIGURATION ====\n");
  printf("Block: %d x %d x %d   Time: %.3f ms\n", bestBX, bestBY, bestBZ, bestTime);

  // Read back GPU results

  cudaEventRecord(start);
  checkCudaErrors( cudaMemcpy(h_u2, d_u1, bytes, cudaMemcpyDeviceToHost) );
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Copy u2 to host: %.1f (ms) \n\n", milli);

  // error check

  float err = 0.0;

  for (k=0; k<NZ; k++) {
    for (j=0; j<NY; j++) {
      for (i=0; i<NX; i++) {
        ind = i + j*NX + k*NX*NY;
        err += (h_u1[ind]-h_u2[ind])*(h_u1[ind]-h_u2[ind]);
      }
    }
  }

  printf("rms error = %f \n",sqrt(err/ (float)(NX*NY*NZ)));

 // Release GPU and CPU memory

  checkCudaErrors( cudaFree(d_u1) );
  checkCudaErrors( cudaFree(d_u2) );
  free(h_u1);
  free(h_u2);

  cudaDeviceReset();
}


Writing laplace3d_new.cu
